<a href="https://colab.research.google.com/github/thecronus/itub/blob/main/itub_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LTSM usando os últimos 60 dias de historico 

In [ ]:
import math
import pandas_datareader as web 
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential 
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use ('fivethirtyeight')
import plotly.graph_objects as go


In [ ]:
df = pd.read_csv('/content/ITUB3.SA.csv')
df = df.set_index('Date')
df = df.dropna()
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-08-23,13.159392,13.423491,13.031896,13.423491,8.827859,267930.0
2012-08-24,13.204926,13.428044,13.018236,13.268674,8.726044,226862.0
2012-08-27,13.268674,13.273228,13.004576,13.214033,8.690110,488643.0
2012-08-28,13.136625,13.218587,13.104751,13.204926,8.684124,72912.0
2012-08-29,13.286888,13.323315,13.022789,13.022789,8.564338,376859.0
...,...,...,...,...,...,...
2021-08-16,27.809999,28.110001,27.450001,27.450001,27.450001,1370200.0
2021-08-17,27.459999,27.680000,27.020000,27.490000,27.490000,888600.0
2021-08-18,27.430000,28.139999,27.280001,27.600000,27.600000,1407900.0


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.High,
                    mode='lines',
                    name='High',
                    marker_color = '#2CA02C',
                    visible = "legendonly"))
fig.add_trace(go.Scatter(x = df.index, y = df.Low,
                    mode='lines',
                    name='Low',
                    marker_color = '#D62728',
                    visible = "legendonly"))
fig.add_trace(go.Scatter(x = df.index, y = df.Open,
                    mode='lines',
                    name='Open',
                    marker_color = '#FF7F0E',
                    visible = "legendonly"))
fig.add_trace(go.Scatter(x = df.index, y = df.Close,
                    mode='lines',
                    name='Close',
                    marker_color = '#1F77B4'))

fig.update_layout(
    title='Preço de fechamento',
    titlefont_size = 28,
    
    xaxis = dict(
        title='Data',
        titlefont_size=16,
        tickfont_size=14),
    
    height = 500,
    
    yaxis=dict(
        title='Preço em R$',
        titlefont_size=16,
        tickfont_size=14),
    
    legend=dict(
        y=0,
        x=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))

fig.show()


In [ ]:
#Criando um novo dataframe com o preço de fechamento 
data = df.filter(['Close'])
#Convertendo o dataframe para um array
dataset = data.values
#Pegando os dados de treinamento 
training_data_len = math.ceil(len(dataset) * .8)
training_data_len

1781

In [ ]:
#Scalonando os dados
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.07339098],
       [0.06622057],
       [0.06368985],
       ...,
       [0.72998135],
       [0.71701304],
       [0.72257093]])

In [ ]:
#Criando dados de treino
train_data = scaled_data[0:training_data_len, :]
#Separando dados de treino X e Y
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i <= 61:
        print(x_train)
        print(y_train)
        print()

[array([0.07339098, 0.06622057, 0.06368985, 0.06326806, 0.05483231,
       0.05145805, 0.04829462, 0.05462144, 0.04428764, 0.0400698 ,
       0.05272339, 0.05799575, 0.06158093, 0.06748595, 0.07486723,
       0.08013954, 0.07296918, 0.06959488, 0.07634349, 0.07275826,
       0.08393565, 0.07971775, 0.0499818 , 0.05483231, 0.04281139,
       0.0259399 , 0.02889241, 0.03268851, 0.02868154, 0.03311031,
       0.03500836, 0.03627369, 0.02108938, 0.00843574, 0.01476256,
       0.01687149, 0.02108938, 0.01644969, 0.01919133, 0.01075559,
       0.00949021, 0.        , 0.01159918, 0.01855866, 0.01476256,
       0.00822487, 0.02151118, 0.01223184, 0.04112426, 0.03416477,
       0.04639657, 0.02952513, 0.01855866, 0.02108938, 0.01898041,
       0.02636169, 0.02762708, 0.03268851, 0.05778483, 0.05546503])]
[0.053145182360084364]

[array([0.07339098, 0.06622057, 0.06368985, 0.06326806, 0.05483231,
       0.05145805, 0.04829462, 0.05462144, 0.04428764, 0.0400698 ,
       0.05272339, 0.05799575, 0.0

In [ ]:
#Convertendo os dados de treino para arrays
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(1721, 60, 1)

In [ ]:
#Construindo o modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
#Compilando o modelo
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, batch_size = 1, epochs = 1)

1721/1721 [==============================] - 54s 30ms/step - loss: 0.0027


In [ ]:
#Criando os dados de teste
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range (60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rsme = np.sqrt(np.mean(predictions - y_test) ** 2)
rsme

0.1500609993973292

In [ ]:
#Plotando os dados
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Vizualinzando os dados
fig = go.Figure()
fig.add_trace(go.Scatter(x = train.index, y = train.Close,
                    mode='lines',
                    name='Treino',
                    marker_color = '#1F77B4'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Close,
                    mode='lines',
                    name='Real',
                    marker_color = '#FF7F0E'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Predictions,
                    mode='lines',
                    name='Predição',
                    marker_color = '#2CA02C'))

fig.update_layout(
    title='Modelo ITUB3',
    titlefont_size = 28,
    hovermode = 'x',
    xaxis = dict(
        title='Data',
        titlefont_size=16,
        tickfont_size=14),
    
    height = 500,
    
    yaxis=dict(
        title='Preço de fechamento em R$',
        titlefont_size=16,
        tickfont_size=14),
    legend=dict(
        y=0,
        x=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))

fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Mostrando os valores real e os preditos 
valid

,Close,Predictions
Date,,
2019-10-29,31.370001,30.405190
2019-10-30,31.559999,30.599773
2019-10-31,31.170000,30.760212
2019-11-01,31.190001,30.836449
2019-11-04,31.330000,30.866865
...,...,...
2021-08-16,27.450001,27.602985
2021-08-17,27.490000,27.569172
2021-08-18,27.600000,27.518021


In [ ]:
data1 = pd.read_csv('/content/ITUB3.SA.csv')
data1 = data1.set_index('Date')
data1 = data1.dropna()
data = data1.filter(['Close'])



In [ ]:


last_60_days = data[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1], 1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print ('Prevendo o preço para o próximo dia')
print()
print(pred_price)

Prevendo o preço para o próximo dia

[[27.356527]]
